In [139]:

import datetime as dt
import pyomo.environ as pyo
import pandas as pd
import os.path

In [130]:
# Datum entry
datum="2021 11 24" # provide year, month, day as a script with spaces or select from the calendar
datum = dt.datetime.strptime(datum, '%Y %m %d').date()
today = dt.date.today()


if datum<today:
    print("You selected a day in the past.")


In [390]:
## inputs we can create a table or sth similar

# department ids
deps=['CH1', 'CH2', 'CH3', 'HT', 'IM1', 'IM2', 'OR1', 'OR2']

# Departments capacities
deps_cap=[660, 435, 600,260,0, 825, 930,0]

dep_df=pd.DataFrame(zip(deps,deps_cap), columns=['dep_name','dep_cap'])

# operation room ids
op_rooms_id={0:"HLK1",1:"HLK2",2:"OP1",3:"OP2",4:"OP3",5:"OP4",6:"OP5",7:"OP6"}
#op_rooms_id=[0,1,2,3,4,5,6,7]

# Operation rooms capacities
op_rooms_cap=[435, 390, 555, 555 ,555,435,435,320]

op_df=pd.DataFrame(zip(op_rooms_id.values(),op_rooms_cap), columns=["op_room_name","op_room_cap"])



In [391]:
op_df

,op_room_name,op_room_cap
0,HLK1,435
1,HLK2,390
2,OP1,555
3,OP2,555
4,OP3,555
5,OP4,435
6,OP5,435
7,OP6,320


In [392]:
dep_df

,dep_name,dep_cap
0,CH1,660
1,CH2,435
2,CH3,600
3,HT,260
4,IM1,0
5,IM2,825
6,OR1,930
7,OR2,0


In [394]:

if os.path.isfile(f'{datum}.csv'):   
    print("it is there")
    basket=pd.read_csv(f'{datum}.csv', index_col="operation_id")
else:
    # we create an empty dataframe only first time
    basket=pd.DataFrame(columns=['Abteilungs-ID','ops','weight','value',"op_room_id"])
    basket.index.name = 'operation_id'

#output
basket.tail()

it is there


,Abteilungs-ID,ops,weight,value,op_room_id
operation_id,,,,,
op_or1_07,CH1,xxxx,70,3,OP2


In [395]:
# We populate basket with 5 cells: index, Abteilung-ID, ops, value, weight
# Every entry will be recorded in this table. 


ind="op_or1_01" # this id can be provided by the departments
basket.loc[ind,'ops']=str("xxxx") # codes of the operations to be done
basket.loc[ind,'Abteilungs-ID']=str('CH1') # name the Department who is asking for the operation
basket.loc[ind,'value']=int(3) # importance or the urgency of the operation
basket.loc[ind, 'weight']=int(90) # in minutes



In [396]:
# show last 5 entries (output)
basket.tail()

,Abteilungs-ID,ops,weight,value,op_room_id
operation_id,,,,,
op_or1_07,CH1,xxxx,70.0,3.0,OP2
op_or1_01,CH1,xxxx,90.0,3.0,NaN


In [397]:
# output
basket_weight=sum(basket['weight'])
print(basket_weight)

160.0


In [373]:


#Create the data for the optimization
data = {}

w = list(basket['weight'])
v = list(basket['value'])
d= list(basket['Abteilungs-ID'])
l=list(basket.index)
items=list(zip(l,d,w,v)) # item_no, weight, value, department


data['items'] = items


data['bins'] = list(zip(op_rooms_id.keys(),op_rooms_cap))
data['bin_capacities'] = op_rooms_cap



data['deps'] = list(zip(deps,deps_cap))
data['dep_capacities'] = deps_cap


In [374]:
data

{'items': [('op_or1_07', 'CH1', 70, 3)],
 'bins': [(0, 435),
  (1, 390),
  (2, 555),
  (3, 555),
  (4, 555),
  (5, 435),
  (6, 435),
  (7, 320)],
 'bin_capacities': [435, 390, 555, 555, 555, 435, 435, 320],
 'deps': [('CH1', 660),
  ('CH2', 435),
  ('CH3', 600),
  ('HT', 260),
  ('IM1', 0),
  ('IM2', 825),
  ('OR1', 930),
  ('OR2', 0)],
 'dep_capacities': [660, 435, 600, 260, 0, 825, 930, 0]}

In [375]:
list(k[0] for k in data['bins'])

[0, 1, 2, 3, 4, 5, 6, 7]

In [376]:
# multi-knapsack, integer divisible

mdl = pyo.ConcreteModel()

# sets
mdl.invs = pyo.Set(initialize=data['items'])
mdl.bins = pyo.Set(initialize=list(k[0] for k in data['bins']))
mdl.deps = pyo.Set(initialize=list(k[0] for k in data['deps']))

# params
mdl.value   = pyo.Param(mdl.invs, initialize= {k:k[3] for k in data['items']})
mdl.weight  = pyo.Param(mdl.invs, initialize= {k:k[2] for k in data['items']})
mdl.bin_cap = pyo.Param(mdl.bins, initialize= {k[0]:k[1] for k in data['bins']})
mdl.dep_cap = pyo.Param(mdl.deps, initialize= {k[0]:k[1] for k in data['deps']})



In [377]:
list(mdl.invs)

[('op_or1_07', 'CH1', 70, 3)]

In [378]:
# vars
mdl.X = pyo.Var(mdl.invs, mdl.bins, within=pyo.Binary)     # the amount from invoice i in bin j


In [379]:
list(mdl.X)

[('op_or1_07', 'CH1', 70, 3, 0),
 ('op_or1_07', 'CH1', 70, 3, 1),
 ('op_or1_07', 'CH1', 70, 3, 2),
 ('op_or1_07', 'CH1', 70, 3, 3),
 ('op_or1_07', 'CH1', 70, 3, 4),
 ('op_or1_07', 'CH1', 70, 3, 5),
 ('op_or1_07', 'CH1', 70, 3, 6),
 ('op_or1_07', 'CH1', 70, 3, 7)]

In [380]:

### Objective ###

mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                        i in mdl.invs for
                        b in mdl.bins), sense=pyo.maximize)



In [381]:
### constraints ###

# don't overstuff bin
def bin_limit(self, b):
    return sum(mdl.X[i, b]*int(i[2]) for i in mdl.invs) <= mdl.bin_cap[b]
mdl.c1 = pyo.Constraint(mdl.bins, rule=bin_limit)



In [382]:
# don't overstuff dep
def dep_limit(self, d):
    summe=0
    for b in bins:
        summe+= sum(mdl.X[i,b]*i[2] for i in mdl.invs ) 
    return(summe<= mdl.dep_cap[d])
mdl.c4 = pyo.Constraint(mdl.deps, rule=dep_limit)


In [383]:
# one_item can be only in a single op room.
def one_item(self, i,d,w,v):
    return sum(mdl.X[i,d,w,v,b] for b in mdl.bins) <=1
mdl.c2 = pyo.Constraint(mdl.invs, rule=one_item)

In [384]:

# solve it...
solver = pyo.SolverFactory('glpk')
results = solver.solve(mdl)
#mdl.X.display()
mdl.display()

Model unknown

  Variables:
    X : Size=8, Index=X_index
        Key                            : Lower : Value : Upper : Fixed : Stale : Domain
        ('op_or1_07', 'CH1', 70, 3, 0) :     0 :   1.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 1) :     0 :   0.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 3) :     0 :   0.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 4) :     0 :   0.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 5) :     0 :   0.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 6) :     0 :   0.0 :     1 : False : False : Binary
        ('op_or1_07', 'CH1', 70, 3, 7) :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   3.0

  Constraints:
    c1 : Size=8
 

In [388]:
for i in mdl.X:
    if pyo.value(mdl.X[i])>=1:
        basket.loc[i[0],"op_room_id"]=op_rooms_id[i[3]]

In [389]:
basket.tail(30)

,Abteilungs-ID,ops,weight,value,op_room_id
operation_id,,,,,
op_or1_07,CH1,xxxx,70,3,OP2


In [393]:
basket.to_csv(f'{datum}.csv', index=True)